In [1]:
import requests
import pandas as pd
import numpy as np
import xarray as xr
import sqlite3
import os
from datetime import datetime
import cdsapi
import csv

In [2]:
def requete_sql (requete) :
    try:
        connexion = sqlite3.connect('./data/liste_piezos.db')
        curseur = connexion.cursor()
        print("Connexion réussie à SQLite")
        curseur.execute(requete)
        connexion.commit()
        resultat = curseur.fetchall()
        curseur.close()
        connexion.close()
        print("Connexion SQLite est fermée")
        return resultat
    except sqlite3.Error as error:
        print("Erreur lors du mis à jour dans la table : ", error)

# Fonction pour lancer les requêtes

In [10]:
def get_on_cds_api(dataframe_piezo, piezo, path):
    # Initialisation de la connexion à l'API CDS
    # wait_until_complete = False : le programme Python n'attend pas que l'API réponde (issu de la libraie asyncio)
    # delete = False : le fichier créé sur le serveur source n'est pas supprimé automatiquement 
    
    date_debut = dataframe_piezo.loc[piezo,'date_debut'].year
    date_fin = dataframe_piezo.loc[piezo,'date_fin'].year
    
    # La limite du nombre de données est de 120000. Avec 10 variables, 31 jours, et 12 mois, on n epeut récupérer que 32 années.
    # Si l'interval d'année est supérieur à 32, on réduit la plage de temps à date de fin - 31 ans pour avoir 32 années max.
    if date_fin - date_debut >= 32:
        date_debut = date_fin - 31
    
    c = cdsapi.Client(wait_until_complete=False, delete=False)
    
    # Données à récupérer auprès de l'API
    r = c.retrieve(
            'reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'format': 'netcdf',
                'variable': [
                    '2m_temperature', 'convective_precipitation', 'convective_rain_rate',
                    'convective_snowfall', 'large_scale_rain_rate', 'large_scale_snowfall', 'snowfall',
                    'large_scale_snowfall_rate_water_equivalent', 'potential_evaporation',
                    'total_precipitation',
                ],
                'year': [str(date) for date in (np.arange(date_debut, date_fin))],
                'month': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                ],
                'day': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                    '13', '14', '15',
                    '16', '17', '18',
                    '19', '20', '21',
                    '22', '23', '24',
                    '25', '26', '27',
                    '28', '29', '30',
                    '31',
                ],
                'time': '00:00',
                'area': [
                    df_piezo.loc[piezo,"latitude"], df_piezo.loc[piezo,"longitude"], df_piezo.loc[piezo,"latitude"], df_piezo.loc[piezo,"longitude"],
                ],
            })    
    return r

# Connexion à la bdd piezo
Construction d'un dataframe 'df_piezo' à partir de la requête sur la bdd

In [11]:
region = "NOR"

In [12]:
connexion = sqlite3.connect('./data/liste_piezos.db')
curseur = connexion.cursor()
requete = f"""
    SELECT code_bss, date_debut, date_fin, nb_mesures_piezo, latitude, longitude 
        FROM data_piezo
    INNER JOIN code_region
    ON code_dpt = departement
    WHERE code_region IS '{region}'
    AND date_debut < '2010-07-01'
    AND date_fin > '2010-07-01'
"""

df_piezo = pd.read_sql_query(requete, connexion, index_col='code_bss')
curseur.close()
connexion.close()

# Conversion des dates
df_piezo["date_debut"] = pd.to_datetime(df_piezo["date_debut"])
df_piezo["date_fin"] = pd.to_datetime(df_piezo["date_fin"])

In [13]:
df_piezo

,date_debut,date_fin,nb_mesures_piezo,latitude,longitude
code_bss,,,,,
02516X0071_P3-90,1997-09-15,2020-06-24,8176,48.439666,0.119902
01446X0005_S3-91,2002-03-28,2022-05-23,7129,48.977692,-0.907758
01506X0016_P1,2001-11-22,2022-05-17,5782,49.003128,1.249032
01474X0048_P1,2005-06-30,2022-05-19,5793,49.053115,0.297654
01801X0010_S1,1985-05-09,2022-05-23,9792,48.881863,1.086499
...,...,...,...,...,...
BSS003VETY_X,1995-02-21,2019-06-20,7624,48.730723,0.005066
BSS003VFXY_X,2000-08-24,2020-03-13,6779,48.505098,-0.071441
00955X0066_PZ,2009-09-01,2022-05-23,4406,49.333927,-1.076711


## Lancer les requêtes auprès de l'API
!!! NE LANCER QU'UNE SEULE FOIS POUR NE PAS CREER DES DOUBLONS DE REQUETES !!!

Un dictionnaire est créé avec en clé le code_bss du piezo et en valeur l'id de la requête

Ne pas lancer la cellule ci-dessous si des requetes sont en cours. 

In [14]:
# Parcourir les codes_bss des piezos dans la liste précédemment créé :
dict_bss = {}

In [15]:
go_requete = input("""
            Voulez-vous lancer une série de nouvelles requêtes ? /n 
            Des requêtes pourraient se retrouver en double. /n 
            Pour lancer les requêtes entrez 'req' et entrée. /n
            Sinon, appuyez simplement sur entrée.
""")
                   
if go_requete == 'req' :
                      
    for piezo in df_piezo.index:
        path_piezo = 'data/era5/' + piezo
        path_nc = f"./{path_piezo}.nc"
        path_pkl = f"{path_piezo}.pkl"

        if os.path.isfile(path_nc) : #si le fichier nc existe on le converti en pkl
            print(f"Le fichier {path_nc} existe.")
            df = xr.open_dataset(path_nc).to_dataframe().reset_index().to_pickle(path=path_pkl)
            print(f"Fichier {piezo}.nc converti en fichier pickle")
            os.remove(path_nc)
            print(f"Fichier {path_nc} supprimé")

        elif os.path.isfile(path_pkl) : #si le fichier pkl existe, on affiche qu'il existe
            print(f"Le fichier {piezo}.pkl existe déjà.")
            # puis on le supprime de la liste
            df_piezo.drop(index=piezo, axis=1)

        else : # Sinon on lance une requête auprès de l'API
            result = get_on_cds_api(df_piezo, piezo, path_nc)
            # récupération de l'id de la requête
            dict_bss[piezo] = result.reply["request_id"]
            print(f"{datetime.now()} : chargement du fichier {piezo}.nc en cours ({dict_bss[piezo]}).")
            
            # Enregistrement de l'id de requete dans un fichier csv
            if os.path.isfile('./data/hist_requetes.csv'): # Si le fichier existe                
                with open('./data/hist_requetes.csv', 'a', encoding='utf-8', newline='') as f:  
                    writer = csv.writer(f, delimiter = ";")
                    for k, v in dict_bss.items():
                        writer.writerow((k, v))
            else : # Si le fichier n'existe pas
                with open('./data/hist_requetes.csv', 'w', encoding='utf-8', newline='') as f:  
                    writer = csv.writer(f, delimiter = ";")
                    writer.writerow(["code_bss", "id_requete"])
                    for k, v in dict_bss.items():
                        writer.writerow((k, v))
else : 
    print("Aucune requête n'a été lancée.")


            Voulez-vous lancer une série de nouvelles requêtes ? /n 
            Des requêtes pourraient se retrouver en double. /n 
            Pour lancer les requêtes entrez 'req' et entrée. /n
            Sinon, appuyez simplement sur entrée.
req


2022-07-28 08:52:55,542 INFO Welcome to the CDS
2022-07-28 08:52:55,542 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-28 08:52:55,650 INFO Welcome to the CDS
2022-07-28 08:52:55,650 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-28 08:52:55,742 INFO Welcome to the CDS
2022-07-28 08:52:55,758 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


2022-07-28 08:52:55.619921 : chargement du fichier 02516X0071_P3-90.nc en cours (f497fb76-830b-4230-8924-7a2db35b8c9a).
2022-07-28 08:52:55.716643 : chargement du fichier 01446X0005_S3-91.nc en cours (ec86c08f-f5fc-4e6f-b859-be066db0f422).


2022-07-28 08:52:55,867 INFO Welcome to the CDS
2022-07-28 08:52:55,868 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-28 08:52:55,933 INFO Welcome to the CDS
2022-07-28 08:52:55,933 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


2022-07-28 08:52:55.831179 : chargement du fichier 01506X0016_P1.nc en cours (38fbb010-068a-48a5-9e24-baa0cfbb9bc4).
2022-07-28 08:52:55.908613 : chargement du fichier 01474X0048_P1.nc en cours (811f0cda-9493-4759-bd79-16a8b64b7b24).
2022-07-28 08:52:56.016188 : chargement du fichier 01801X0010_S1.nc en cours (e4e8efe0-5fc6-4127-ba5c-e12696757db9).


...

2022-07-28 08:53:11.103731 : chargement du fichier BSS003UKJY_X.nc en cours (b3488232-daff-41ab-875b-629ac35e82f2).
2022-07-28 08:53:11.219389 : chargement du fichier BSS003VETY_X.nc en cours (bcf67594-85db-41e2-966b-f9a50124d883).


2022-07-28 08:53:11,422 INFO Welcome to the CDS
2022-07-28 08:53:11,423 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-28 08:53:11,576 INFO Welcome to the CDS


2022-07-28 08:53:11.383958 : chargement du fichier BSS003VFXY_X.nc en cours (b7c1b811-178d-4a70-8d6c-9bbfb95902e2).
2022-07-28 08:53:11.537670 : chargement du fichier 00955X0066_PZ.nc en cours (290f62a8-c135-4362-bae9-87dde5cf0782).


2022-07-28 08:53:11,577 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-28 08:53:11,713 INFO Welcome to the CDS
2022-07-28 08:53:11,713 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


2022-07-28 08:53:11.673286 : chargement du fichier 02891X0007_PZ.nc en cours (5dfff1cc-7a70-47c9-a995-e302f3f7062f).
2022-07-28 08:53:11.829875 : chargement du fichier 02531X0029_PZ.nc en cours (dc4b6bae-e625-46bf-b5fa-0fab9fdf7540).


## Récupérer automatiquement les requêtes abouties 
Peut être lancé à tout moment pour récupérer des fichiers téléchargés 

In [3]:
# Chargement du dictionnaire à partir du csv d'historiques de requetes
with open('./data/hist_requetes.csv', mode='r') as f:
    reader = csv.reader(f, delimiter=";")
    # Permet de sortir la ligne d'en-tête du dictionnaire
    headings = next(reader)
    dict_bss_from_csv = {rows[0]:rows[1] for rows in reader}

nbre_fichier_OK = 0

    
for piezo in dict_bss_from_csv.copy().keys():
    path_piezo = 'data/era5/' + piezo
    path_nc = f"./{path_piezo}.nc"
    path_pkl = f"{path_piezo}.pkl"

    if not os.path.isfile(path_nc) :
        try:
            reply = dict(request_id=dict_bss_from_csv[piezo]) # request_id from above
            new_client = cdsapi.Client(wait_until_complete=False, delete=False)
            result = cdsapi.api.Result(new_client, reply)
            result.update()
            reply = result.reply
            if reply['state'] == 'completed':
                nbre_fichier_OK += 1
                result.download(path_nc)
                print(f"{datetime.now()} : Fichier {piezo}.nc téléchargé.")
                df = xr.open_dataset(path_nc).to_dataframe().reset_index().to_pickle(path=path_pkl)
                print(f"Fichier {piezo}.nc converti en fichier pickle")
                os.remove(path_nc)
                print(f"Fichier {path_nc}.nc supprimé")
                # Si le fichier a été téléchargé on le supprime de la liste du dictionnaire
                del dict_bss[piezo]
        except:
            #print(f"Récupération de l'ID {dict_bss_from_csv[piezo]} impossible.")
            pass
if nbre_fichier_OK == 0:
    print("Aucun fichier de récupéré.")
else :
    print(f"{nbre_fichier_OK} fichiers ont été téléchargés.")

# Mise à jour de la base de donnéees (champ fichier_ERA5)
bss_era5 = pd.Series([_.replace(".pkl","") for _ in os.listdir(r"./data/era5") if _.endswith(r".pkl")], name="era5")
requete_fichier_era5 = f"""
        UPDATE data_piezo
        SET fichier_era5 =
        (CASE WHEN code_bss IN {tuple(bss_era5)}
        THEN 1 ELSE 0 END)
        """
requete_sql(requete_fichier_era5)
print("Base de données à jour")

2022-07-28 08:51:10,352 INFO Downloading https://download-0018.copernicus-climate.eu/cache-compute-0018/cache/data5/adaptor.mars.internal-1658750584.5975208-32198-2-8920aaf5-1c5a-4add-82ed-67c578423c7a.nc to ./data/era5/09145X0057_N210.nc (174.6K)
2022-07-28 08:51:10,684 INFO Downloading https://download-0020.copernicus-climate.eu/cache-compute-0020/cache/data5/adaptor.mars.internal-1658750584.637897-24257-20-37b83f68-a036-48f6-bea6-33ef2cab27e4.nc to ./data/era5/09406X0643_P130.nc (174.6K)
2022-07-28 08:51:10,907 INFO Downloading https://download-0002-clone.copernicus-climate.eu/cache-compute-0002/cache/data7/adaptor.mars.internal-1658750584.7223244-748-17-f82677bd-9e72-4719-a6b6-34952bcfb414.nc to ./data/era5/09145X0059_N340.nc (174.6K)
2022-07-28 08:51:11,154 INFO Downloading https://download-0006-clone.copernicus-climate.eu/cache-compute-0006/cache/data2/adaptor.mars.internal-1658749677.502755-29611-3-76c7cec4-9a15-46a5-8e1d-71f7b3f44100.nc to ./data/era5/09735X0070_P40.nc (114.7K)

2022-07-28 08:51:14.963081 : Fichier 09724X0023_P2.nc téléchargé.
Fichier 09724X0023_P2.nc converti en fichier pickle
Fichier ./data/era5/09724X0023_P2.nc.nc supprimé


2022-07-28 08:51:15,167 INFO Downloading https://download-0021.copernicus-climate.eu/cache-compute-0021/cache/data1/adaptor.mars.internal-1658755720.8252316-20710-15-bfdeaa4f-ff3e-4c1a-bfae-374289f3c386.nc to ./data/era5/09675X0123_P2.nc (140.4K)
2022-07-28 08:51:15,361 INFO Downloading https://download-0011-clone.copernicus-climate.eu/cache-compute-0011/cache/data9/adaptor.mars.internal-1658757864.438635-15785-18-d08a6f82-eada-4c63-b4f0-ba065ddf5370.nc to ./data/era5/09665X0073_203.nc (268.8K)
2022-07-28 08:51:15,616 INFO Downloading https://download-0021.copernicus-climate.eu/cache-compute-0021/cache/data1/adaptor.mars.internal-1658754400.1862056-13003-17-9f7f29ce-f2c3-4766-9b7c-0f1242640969.nc to ./data/era5/09724X0034_P53.nc (106.2K)
2022-07-28 08:51:15,820 INFO Downloading https://download-0001-clone.copernicus-climate.eu/cache-compute-0001/cache/data7/adaptor.mars.internal-1658756363.3723745-30582-9-3b7cbb4f-ec60-40f3-80e8-4786721af4f4.nc to ./data/era5/09951X0068_F3.nc (268.8K)


2022-07-28 08:51:18.509320 : Fichier 09668X0106_16.nc téléchargé.
Fichier 09668X0106_16.nc converti en fichier pickle
Fichier ./data/era5/09668X0106_16.nc.nc supprimé


2022-07-28 08:51:18,721 INFO Downloading https://download-0015-clone.copernicus-climate.eu/cache-compute-0015/cache/data8/adaptor.mars.internal-1658767139.815615-21372-4-c5aa49d8-da1a-4049-8d6d-0a90b5b13fdd.nc to ./data/era5/09994X0400_P-36.nc (114.7K)
2022-07-28 08:51:18,926 INFO Downloading https://download-0002-clone.copernicus-climate.eu/cache-compute-0002/cache/data9/adaptor.mars.internal-1658767204.951801-31369-15-a5deaff2-5a6c-409f-8348-daab4b791f47.nc to ./data/era5/09145X0058_N256.nc (174.6K)
2022-07-28 08:51:19,126 INFO Downloading https://download-0005-clone.copernicus-climate.eu/cache-compute-0005/cache/data5/adaptor.mars.internal-1658767203.8048234-27135-5-b02f1ffb-02f3-4ce5-ab9f-cc087c2cc616.nc to ./data/era5/08465X0020_S6.nc (149K)
2022-07-28 08:51:19,327 INFO Downloading https://download-0004-clone.copernicus-climate.eu/cache-compute-0004/cache/data9/adaptor.mars.internal-1658765027.8783553-6024-1-b169bf8a-702c-4d93-bb44-08ff4f53c8dd.nc to ./data/era5/09724X0028_P37.nc 

...

2022-07-28 08:51:27.794691 : Fichier 10247X0096_P.nc téléchargé.
Fichier 10247X0096_P.nc converti en fichier pickle
Fichier ./data/era5/10247X0096_P.nc.nc supprimé


2022-07-28 08:51:28,050 INFO Downloading https://download-0011-clone.copernicus-climate.eu/cache-compute-0011/cache/data1/adaptor.mars.internal-1658791861.0741184-24920-16-4315712b-06a1-41bb-8a3b-ca3e861694af.nc to ./data/era5/09436X0138_1P.nc (268.8K)
2022-07-28 08:51:28,259 INFO Downloading https://download-0018.copernicus-climate.eu/cache-compute-0018/cache/data5/adaptor.mars.internal-1658789809.1595438-16743-13-72688901-462d-4e30-a88e-ff02e8ccb22a.nc to ./data/era5/09994X0533_PZ_GDP.nc (114.7K)
2022-07-28 08:51:28,459 INFO Downloading https://download-0011-clone.copernicus-climate.eu/cache-compute-0011/cache/data5/adaptor.mars.internal-1658790209.454544-28593-11-010d69af-42dd-4a70-90df-2b940784f6de.nc to ./data/era5/09998X0197_PZ_HIP.nc (106.2K)
2022-07-28 08:51:28,659 INFO Downloading https://download-0010-clone.copernicus-climate.eu/cache-compute-0010/cache/data5/adaptor.mars.internal-1658791715.0813882-7322-4-61a04755-8e4d-466c-9fdc-9f479ba82fc0.nc to ./data/era5/09975X0009_FR.n

2022-07-28 08:51:30.797517 : Fichier 09994X0537_PZ_ROU.nc téléchargé.
Fichier 09994X0537_PZ_ROU.nc converti en fichier pickle
Fichier ./data/era5/09994X0537_PZ_ROU.nc.nc supprimé


2022-07-28 08:51:31,050 INFO Downloading https://download-0017.copernicus-climate.eu/cache-compute-0017/cache/data9/adaptor.mars.internal-1658800523.6146288-6711-12-db94a5e4-6fc5-4bd1-9728-9d0bc8a6b114.nc to ./data/era5/09146X0074_PU.nc (268.8K)
2022-07-28 08:51:31,265 INFO Downloading https://download-0014-clone.copernicus-climate.eu/cache-compute-0014/cache/data1/adaptor.mars.internal-1658796837.1501489-24744-5-9ea49222-a767-44f8-8afa-04c05c211370.nc to ./data/era5/09735X0224_PZS10V.nc (46.2K)
2022-07-28 08:51:31,502 INFO Downloading https://download-0017.copernicus-climate.eu/cache-compute-0017/cache/data2/adaptor.mars.internal-1658796878.0729573-8914-3-89689476-d9cb-410b-af1a-12dbd65330a7.nc to ./data/era5/09735X0215_PZS9AV.nc (106.2K)
2022-07-28 08:51:31,681 INFO Downloading https://download-0021.copernicus-climate.eu/cache-compute-0021/cache/data9/adaptor.mars.internal-1658796282.988017-32193-14-9f60916f-c38a-4101-8c14-66e925fa69dc.nc to ./data/era5/09994X0532_PZ_VAV.nc (114.7K)


103 fichiers ont été téléchargés.
Connexion réussie à SQLite
Connexion SQLite est fermée
Base de données à jour


## Jointure era5 et piezo

In [4]:
bss_era5 = pd.Series([_.replace(".pkl","") for _ in os.listdir(r"./data/era5") if _.endswith(r".pkl")], name="era5")
bss_piezo = pd.Series([_.replace(".csv","") for _ in os.listdir(r"./data/piezo") if _.endswith(r".csv")], name="piezo")
piezos_communs = np.intersect1d(bss_era5, bss_piezo)

path = "./data/combined_data/"
# Le fichier doit avoir un minimum de 100 mesures
nb_min_val_com = 100

for piezo in piezos_communs :
    if not os.path.isfile(f'{path}/comb_{piezo}.csv') : #si le fichier n'existe pas 
        df_piezo = pd.read_csv(f"./data/piezo/{piezo}.csv", sep = ";", index_col=0, parse_dates=True)
        # Group_by date_mesure dans le cas de plusieurs mesures le même jour ce qui provoquerait une erreur
        df_piezo = pd.DataFrame(df_piezo.groupby("date_mesure").mean())
        df_piezo = df_piezo.resample('1D').asfreq()
        df_era5 = pd.read_pickle(f"./data/era5/{piezo}.pkl").set_index('time').drop(["latitude", "longitude"], axis=1)
        if "expver" in df_era5.columns :
            df_era5 = df_era5.loc[df_era5["expver"]==1,:].drop("expver", axis=1)
        df = pd.merge(df_piezo, df_era5, how='inner', left_index=True, right_index=True)
        nb_com_obs = df.shape[0]
        if nb_com_obs >= nb_min_val_com :
            df.to_csv(f"data/combined_data/comb_{piezo}.csv" ,sep=";", index=True)
            print(f"Le fichier comb_{piezo}.csv a été créé et contient {nb_com_obs} obseravtions.")
        else :
            print(f"Le piezo {piezo} ne contient pas assez de données (seulement {nb_com_obs}).")
    else : 
        #print(f"Le fichier comb_{piezo}.csv existe déjà.")
        pass
        
# Mise à jour bdd
requete_fichier_combined = f"""
        UPDATE data_piezo
        SET fichier_combined =
        (CASE WHEN code_bss IN {tuple(piezos_communs)}
        THEN 1 ELSE 0 END)
        """
requete_sql(requete_fichier_combined)
print("Base de données à jour")

Le fichier comb_09172X0094_P.csv a été créé et contient 11323 obseravtions.
Le fichier comb_09432X0005_P.csv a été créé et contient 11323 obseravtions.
Le fichier comb_09668X0106_16.csv a été créé et contient 11323 obseravtions.
Le fichier comb_09724X0023_P2.csv a été créé et contient 11323 obseravtions.
Le fichier comb_09994X0537_PZ_ROU.csv a été créé et contient 4318 obseravtions.
Le fichier comb_10247X0096_P.csv a été créé et contient 5467 obseravtions.
Connexion réussie à SQLite
Connexion SQLite est fermée
Base de données à jour


# Récupération manuelle

In [ ]:
reply = dict(request_id="54c52ee3-eee4-481d-b72e-fecd315085cb")
new_client = cdsapi.Client(wait_until_complete=False, delete=False)
result = cdsapi.api.Result(new_client, reply)
result.update()
reply = result.reply
if reply['state'] == 'completed':
    result.download('file.nc')

In [ ]:
requete_fichier_era5 = f"""
        UPDATE data_piezo
        SET fichier_era5 =
        (CASE WHEN code_bss IN {tuple(bss_era5)}
        THEN 1 ELSE 0 END)
        """
requete_sql(requete_fichier_era5)

In [ ]:
update table set
    columnx = (case when condition then 25 else columnx end)